In [ ]:
!pip install ultralytics

In [1]:
from datasets import convert_cvat_to_yolo

BATCH_SIZE = 16
EPOCHS = 150
PATIENCE = 10

param_grid = {
    'img_size': [1024, 512],
    'pretrained_model': ['yolov8n.pt', 'yolov8s.pt', 'yolo11n.pt', 'yolo11s.pt'],
    'dataset': [
        convert_cvat_to_yolo([
            ('./datasets/synthetic_gray/annotations.xml', 'train', None, None),
            ("./datasets/clear1/annotations.xml", 'val', None, None),
            ("./datasets/clear2/annotations.xml", 'val', None, None),
            ('./datasets/clear3/annotations.xml', 'val', None, None),
        ], './datasets/.yolo.0'),
        convert_cvat_to_yolo([
            ('./datasets/30/annotations.xml', 'train', 'PNG', ['clear']),
            ('./datasets/60/annotations.xml', 'train', 'PNG', ['clear']),
            ('./datasets/90/annotations.xml', 'train', 'PNG', ['clear']),
            ('./datasets/090/annotations.xml', 'train', 'PNG', ['clear']),
            ('./datasets/190/annotations.xml', 'train', 'PNG', ['clear']),
            ("./datasets/clear1/annotations.xml", 'val', None, None),
            ("./datasets/clear2/annotations.xml", 'val', None, None),
            ('./datasets/clear3/annotations.xml', 'val', None, None),
        ], './datasets/.yolo.1'),
        convert_cvat_to_yolo([
            ('./datasets/30/annotations.xml', 'train', 'PNG', ['elongated', 'distinguishable', 'inferred']),
            ('./datasets/60/annotations.xml', 'train', 'PNG', ['elongated', 'distinguishable', 'inferred']),
            ('./datasets/90/annotations.xml', 'train', 'PNG', ['elongated', 'distinguishable', 'inferred']),
            ('./datasets/090/annotations.xml', 'train', 'PNG', ['elongated', 'distinguishable', 'inferred']),
            ('./datasets/190/annotations.xml', 'train', 'PNG', ['elongated', 'distinguishable', 'inferred']),
            ('./datasets/30/annotations.xml', 'val', 'PNG', ['clear']),
            ('./datasets/60/annotations.xml', 'val', 'PNG', ['clear']),
            ('./datasets/90/annotations.xml', 'val', 'PNG', ['clear']),
            ('./datasets/090/annotations.xml', 'val', 'PNG', ['clear']),
            ('./datasets/190/annotations.xml', 'val', 'PNG', ['clear']),
        ], './datasets/.yolo.2'),
        convert_cvat_to_yolo([
            ('./datasets/30/annotations.xml', 'train', 'PNG', None),
            ('./datasets/60/annotations.xml', 'train', 'PNG', None),
            ('./datasets/90/annotations.xml', 'train', 'PNG', None),
            ('./datasets/090/annotations.xml', 'train', 'PNG', None),
            ('./datasets/190/annotations.xml', 'train', 'PNG', None),
            ('./datasets/simple/annotations.xml', 'train', None, None),
            ('./datasets/clear3/annotations.xml', 'train', None, None),
            ('./datasets/clear1/annotations.xml', 'val', None, None),
            ('./datasets/clear2/annotations.xml', 'val', None, None),
        ], './datasets/.yolo.3')
    ]
}

Wrote 960 samples for split 'train'.
Wrote 66 samples for split 'val'.
Wrote 0 samples for split 'test'.


In [ ]:
import itertools
import ultralytics


def grid_search(params):
    keys = list(params.keys())
    for values in itertools.product(*(params[key] for key in keys)):
        yield dict(zip(keys, values))


metrics = []

for params in grid_search(param_grid):
    print('Training...')
    print(params)
    model = ultralytics.YOLO(params['pretrained_model'])
    result = model.train(data=params['dataset'], imgsz=params['img_size'], epochs=EPOCHS, batch=BATCH_SIZE, patience=PATIENCE,
                         plots=True)

    metrics.append((params, model.val()))

In [ ]:
import pandas as pd
import json

df = pd.DataFrame([
    {
        "ImageSize": p['img_size'],
        "PretrainedModel": p['pretrained_model'],
        "Dataset": p['dataset'][-18:],
        "Precision": v.box.p[0] * 100,
        "Recall": v.box.r[0] * 100,
        "F1Score": v.box.f1[0] * 100,
        'Speed': sum(v.speed.values()) if hasattr(v, 'speed') else None,
        **{f"AP@{f/100:.2f}": v.box.all_ap[0][i] * 100 for i, f in enumerate(range(50, 100, 5))}
    } for p, v in metrics
])

print(df)
print(json.dumps(df.to_dict(orient='list')))

In [ ]:
samples = []

for i, (p, v) in enumerate(metrics):
    s = [{
        "index": i,
        "name": name,
        "x":x.tolist(),
        "y":y.tolist(),
        "x_label":x_label,
        "y_label":y_label
    } for name, (x, (y,), x_label, y_label) in zip(v.curves, v.curves_results)]

    for e in s:
        samples.append(e)

df2 = pd.DataFrame(samples)

with open('curves.json', 'w') as file:
    json.dump(df2.to_dict(orient='list'), file)